In [1]:
pip install -q yt-dlp decord langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 68.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.7 which is incompatible.
langchain-text-splitters 0.3.9 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!gdown https://drive.google.com/uc?id=1sq86cC0oBBLLcz1T4YTXEkL4PwmTm6tu

Downloading...
From: https://drive.google.com/uc?id=1sq86cC0oBBLLcz1T4YTXEkL4PwmTm6tu
To: /kaggle/working/cookies.txt
100%|████████████████████████████████████████| 797k/797k [00:00<00:00, 88.3MB/s]


In [3]:
import yt_dlp
import subprocess
import os
import google.generativeai as genai
import PIL.Image
import io

def extract_audio_from_mp4(video_path, output_audio_path):
  try:
      command = [
          'ffmpeg',
          '-i', video_path,           # Input video
          '-vn',                       # Không lấy video
          '-acodec', 'libmp3lame',    # Codec MP3
          '-q:a', '2',                 # Chất lượng audio (0-9, 2 là tốt)
          '-y',                        # Overwrite nếu file đã tồn tại
          output_audio_path
      ]

      subprocess.run(command, check=True, capture_output=True)
      print(f"✓ Đã tách audio: {output_audio_path}")
      return True
  except subprocess.CalledProcessError as e:
      print(f"✗ Lỗi khi tách audio: {e}")
      return False


def download_and_extract_audio(video_path, output_name):
  """
  video_path: link video
  """
  video_file = f"{output_name}.mp4"
  audio_file = f"{output_name}.mp3"

  ydl_opts = {
    'format': 'best[ext=mp4]',
    'outtmpl': video_file,
    'cookiefile': '/kaggle/working/cookies.txt',
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([ytb_path])

  extract_audio_from_mp4(video_file, audio_file)


In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Iterable, Sequence

def _iter_batches(items, batch_size):
    for i in range(0, len(items), batch_size):
        yield items[i:i + batch_size]


def _playlist_items_arg(start_index: int | None, end_index: int | None) -> str | None:
    """Build yt-dlp playlist_items string from start/end index."""
    if start_index is None and end_index is None:
        return None
    start = start_index if start_index is not None else 1
    if end_index is None:
        return f"{start}-"
    return f"{start}-{end_index}"


def _build_playlist_opts(output_dir: str, subtitle_langs: Sequence[str], playlist_items: str | None = None) -> dict:
    opts = {
        'format': 'bestvideo+bestaudio/best',
        'merge_output_format': 'mp4',
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': list(subtitle_langs),
        'subtitlesformat': 'vtt',
        'ignoreerrors': True,
        'cookiefile': '/kaggle/working/cookies.txt',
        'paths': {
            'home': output_dir,
            'temp': output_dir,
        }
    }
    if playlist_items:
        opts['playlist_items'] = playlist_items
    return opts


def _download_playlist_entry(entry: dict, base_opts: dict, output_dir: str):
    idx = int(entry['playlist_index']) if entry.get('playlist_index') else 0
    opts = base_opts.copy()
    opts['outtmpl'] = os.path.join(
        output_dir,
        f"{idx:03d} - %(title)s.%(ext)s"
    )

    filepath = None
    with yt_dlp.YoutubeDL(opts) as ydl:
        info = ydl.extract_info(entry['webpage_url'], download=True)
        filepath = ydl.prepare_filename(info)
        merge_ext = opts.get('merge_output_format')
        if merge_ext:
            filepath = os.path.splitext(filepath)[0] + f".{merge_ext}"

    return idx, entry.get('title', 'unknown title'), filepath


def download_playlist_batch(
    playlist_url: str,
    output_dir: str = '/kaggle/working/playlist_downloads',
    subtitle_langs: Sequence[str] = ('vi',),
    batch_size: int = 4,
    start_index: int | None = None,
    end_index: int | None = None,
):
    """Download a YouTube playlist in sequential batches (default batch size 4)."""
    if batch_size < 1:
        raise ValueError('batch_size must be >= 1')

    os.makedirs(output_dir, exist_ok=True)

    # Build playlist_items filter
    items_filter = _playlist_items_arg(start_index, end_index)
    
    # Fetch playlist metadata without downloading first
    metadata_opts = {
        'ignoreerrors': True,
        'cookiefile': '/kaggle/working/cookies.txt',
    }
    if items_filter:
        metadata_opts['playlist_items'] = items_filter
    
    with yt_dlp.YoutubeDL(metadata_opts) as ydl:
        playlist_info = ydl.extract_info(playlist_url, download=False)

    entries: Iterable[dict] = playlist_info.get('entries', [])
    filtered_entries = []
    fallback_index = start_index if start_index is not None else 1
    for entry in entries:
        if not entry:
            fallback_index += 1
            continue
        if entry.get('playlist_index') is None:
            entry['playlist_index'] = fallback_index
        filtered_entries.append(entry)
        fallback_index += 1

    base_opts = _build_playlist_opts(output_dir, subtitle_langs, items_filter)

    results = []
    for chunk in _iter_batches(filtered_entries, batch_size):
        with ThreadPoolExecutor(max_workers=len(chunk)) as executor:
            future_map = {
                executor.submit(_download_playlist_entry, entry, base_opts, output_dir): entry
                for entry in chunk
            }
            for future in as_completed(future_map):
                entry = future_map[future]
                try:
                    idx, title, video_path = future.result()
                    print(f"✓ Downloaded #{idx:03d}: {title}")
                    results.append({'index': idx, 'title': title, 'video_path': video_path})
                except Exception as exc:
                    print(f"✗ Failed #{entry.get('playlist_index')}: {entry.get('title')} -> {exc}")

    return sorted(results, key=lambda item: item['index'])


In [ ]:
playlist_url = "https://www.youtube.com/playlist?list=PLM1poDh0i5QS8klF38_bHsXmZS4smbHse"
playlist_output_dir = "/kaggle/working/playlist_downloads"
subtitle_langs = ("vi",)
batch_size = 4
start_index = 10   # chỉ số video đầu tiên muốn tải (None nếu bỏ qua)
end_index = 40  # chỉ số video cuối muốn tải (None để tới hết playlist)

summary = download_playlist_batch(
    playlist_url,
    output_dir=playlist_output_dir,
    subtitle_langs=subtitle_langs,
    batch_size=batch_size,
    start_index=start_index,
    end_index=end_index,
)

if summary:
    download_first = summary[0]['index']
    download_last = summary[-1]['index']
else:
    download_first = start_index if start_index is not None else "-"
    download_last = end_index if end_index is not None else "-"

print(
    f"\nĐã tải {len(summary)} video (index {download_first}→{download_last})"
    f" vào thư mục {playlist_output_dir}"
)


In [ ]:
from dataclasses import dataclass
from pathlib import Path
import json
import shutil
import unicodedata
from difflib import SequenceMatcher
import base64

import numpy as np
import cv2
from decord import VideoReader, cpu
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from httpx import Client


@dataclass
class PlaylistItem:
    index: int
    title: str
    video_path: str


def normalized(text):
    if text is None:
        return ""

    s = text.strip()
    s = s.replace('\n', ' ').replace('\r', ' ')
    s = " ".join(s.split())
    s = s.lower()
    s = s.replace(' - ', '-')

    s = unicodedata.normalize('NFD', s)
    s = ''.join(char for char in s if unicodedata.category(char) != 'Mn')

    return s


def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


def is_similar_text(a, b, sim_threshold=0.85):
    na, nb = normalized(a), normalized(b)
    if na == "" and nb == "":
        return True
    sim = similarity(na, nb)
    return sim >= sim_threshold


def numpy_to_base64(img: np.ndarray) -> str:
    _, buffer = cv2.imencode(".png", img)
    img_bytes = buffer.tobytes()
    img_b64 = base64.b64encode(img_bytes).decode("utf-8")
    return img_b64


def get_fpt_llm(temperature=0.0):
    client = Client(verify=False)
    return ChatOpenAI(
        model="Qwen/Qwen2.5-VL-3B-Instruct",
        openai_api_base="https://my-container-4voix9lz-8000.serverless.fptcloud.com/v1",
        openai_api_key="your_api_key",
        http_client=client,
        temperature=temperature,
    )


llm = get_fpt_llm()


class VideoOCRProcessor:
    def __init__(self, video_path: str, crop=(100, 530, 1200, 680)):
        self.video_path = video_path
        self.crop = crop
        self.vr = VideoReader(video_path, ctx=cpu(0))
        self.total_frames = len(self.vr)
        self.video_fps = float(self.vr.get_avg_fps())
        self.ocr_calls = 0
        self.ocr_cache = {}

    def read_frame(self, time_point: float):
        frame_index = int(time_point * self.video_fps) + 1
        frame_index = min(frame_index, self.total_frames - 1)

        if frame_index < 0 or frame_index >= self.total_frames:
            return None

        frame = self.vr[frame_index].asnumpy()
        x1, y1, x2, y2 = self.crop
        return frame[y1:y2, x1:x2]

    def ocr_with_fpt(self, frame_time: float):
        if frame_time in self.ocr_cache:
            return self.ocr_cache[frame_time]

        frame = self.read_frame(frame_time)
        if frame is None:
            return ""

        self.ocr_calls += 1
        img_b64 = numpy_to_base64(frame)

        message = HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": "Hãy đọc OCR trong ảnh này, trả về text đúng nhất. Chỉ trả về text, không giải thích thêm. Nếu trong ảnh không có text hoặc không đọc được ocr thì trả về "". Nếu trong ảnh có phần bảng tên của người phỏng vấn thì cũng không trả về text ở trong đó.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{img_b64}"},
                },
            ]
        )

        response = llm.invoke([message])
        text = response.content
        self.ocr_cache[frame_time] = text
        return text

    def binary_segmentation(self, left_time, right_time, left_text=None, right_text=None, threshold=0.5, sim_threshold=0.9):
        if left_text is None:
            left_text = self.ocr_with_fpt(left_time)
        if right_text is None:
            right_text = self.ocr_with_fpt(right_time)

        if is_similar_text(left_text, right_text, sim_threshold):
            return []

        if right_time - left_time <= threshold:
            return [{"end": int(right_time), "text": left_text}]

        mid_time = (left_time + right_time) / 2
        mid_text = self.ocr_with_fpt(mid_time)

        return (
            self.binary_segmentation(left_time, mid_time, left_text, mid_text, threshold, sim_threshold)
            + self.binary_segmentation(mid_time, right_time, mid_text, right_text, threshold, sim_threshold)
        )

    def scan_video(self, scan_step=4):
        video_duration = int(self.total_frames / self.video_fps)
        timestamps = []

        prev_time = 1
        prev_text = self.ocr_with_fpt(prev_time)

        for t in range(scan_step, video_duration + scan_step, scan_step):
            curr_time = min(t, video_duration - 1)
            curr_text = self.ocr_with_fpt(curr_time)

            if curr_text != prev_text:
                change_times = self.binary_segmentation(prev_time, curr_time, prev_text, curr_text)
                timestamps.extend(change_times)

            prev_time = curr_time
            prev_text = curr_text

        return timestamps

    @staticmethod
    def build_segments(timestamps):
        timestamps_with_start = []
        prev_end = 0.0

        for seg in timestamps:
            start = prev_end
            end = seg['end']
            text = seg['text']
            timestamps_with_start.append({
                "start": start,
                "end": end,
                "text": text
            })
            prev_end = end

        return timestamps_with_start


def cut_audio_segments(
    input_audio_path,
    segments,
    output_folder="audio_segments",
    json_output="segments.json"
):

    os.makedirs(output_folder, exist_ok=True)

    output = []

    for i, seg in enumerate(segments):
        start = seg["start"]
        end = seg["end"]
        text = seg.get("text", "")

        duration = end - start
        audio_path = os.path.join(output_folder, f"{i}.wav")

        cmd = [
            "ffmpeg",
            "-y",
            "-i", input_audio_path,
            "-ss", str(start),
            "-t", str(duration),
            audio_path
        ]

        subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        output.append({
            "id": i,
            "start": start,
            "end": end,
            "text": text,
            "audio_path": audio_path
        })

    with open(json_output, "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    return output


def process_video_item(item: PlaylistItem, base_output_dir: str, scan_step: int = 4):
    video_path = Path(item.video_path)
    output_dir = Path(base_output_dir) / f"{item.index:03d}"
    output_dir.mkdir(parents=True, exist_ok=True)

    audio_path = output_dir / f"{item.index:03d}.mp3"
    extract_audio_from_mp4(str(video_path), str(audio_path))

    processor = VideoOCRProcessor(str(video_path))
    timestamps = processor.scan_video(scan_step=scan_step)
    segments = processor.build_segments(timestamps)

    segments_folder = output_dir / "audio_segments"
    segments_json = output_dir / f"{item.index:03d}_segments.json"
    cut_audio_segments(
        input_audio_path=str(audio_path),
        segments=segments,
        output_folder=str(segments_folder),
        json_output=str(segments_json)
    )

    zip_path = shutil.make_archive(str(segments_folder), 'zip', str(segments_folder))

    return {
        "index": item.index,
        "title": item.title,
        "video_path": str(video_path),
        "audio_path": str(audio_path),
        "segments_json": str(segments_json),
        "segments_zip": zip_path,
        "segment_count": len(segments),
        "ocr_calls": processor.ocr_calls,
    }


def process_playlist_items(
    items,
    base_output_dir: str,
    batch_size: int = 4,
    scan_step: int = 4,
    start_index: int | None = None,
    end_index: int | None = None,
):
    os.makedirs(base_output_dir, exist_ok=True)

    filtered_items = [
        item for item in items
        if (start_index is None or item.index >= start_index)
        and (end_index is None or item.index <= end_index)
    ]

    results = []
    for chunk in _iter_batches(filtered_items, batch_size):
        with ThreadPoolExecutor(max_workers=len(chunk)) as executor:
            future_map = {
                executor.submit(process_video_item, item, base_output_dir, scan_step): item
                for item in chunk
            }
            for future in as_completed(future_map):
                item = future_map[future]
                try:
                    result = future.result()
                    print(f"✓ Processed #{item.index:03d}: {item.title}")
                    results.append(result)
                except Exception as exc:
                    print(f"✗ Failed #{item.index:03d}: {item.title} -> {exc}")

    return sorted(results, key=lambda item: item['index'])

In [ ]:
playlist_items = [
    PlaylistItem(index=item['index'], title=item['title'], video_path=item['video_path'])
    for item in summary
]

print(f"Tổng số video cần xử lý: {len(playlist_items)}")

In [ ]:

processing_output_dir = "/kaggle/working/playlist_processed"
scan_step = 4

processing_results = process_playlist_items(
    playlist_items,
    base_output_dir=processing_output_dir,
    batch_size=batch_size,
    scan_step=scan_step,
    start_index=start_index,
    end_index=end_index,
)

if processing_results:
    processed_first = processing_results[0]['index']
    processed_last = processing_results[-1]['index']
else:
    processed_first = start_index if start_index is not None else "-"
    processed_last = end_index if end_index is not None else "-"

print(
    f"\nHoàn tất xử lý {len(processing_results)} video (index {processed_first}→{processed_last})"
    f" và lưu tại {processing_output_dir}"
)
for item in processing_results[:3]:
    print(f"#{item['index']:03d} | segments: {item['segment_count']} | json: {item['segments_json']}")
